# Constructing Graphs from [AlphaFold Protein Structure Database](https://alphafold.ebi.ac.uk/)

In this quick tutorial we show how to construct graphs from the [AlphaFold Protein Structure Database](https://alphafold.ebi.ac.uk/). This is a fantastic resource for the community and we hope to make it more accessible to the Geometric Deep Learning community!

It's plug and play with the top-level [`graphein.protein.construct_graph()`](https://graphein.ai/modules/graphein.protein.html#graphein.protein.graphs.construct_graph) function; however we now require UniProt identifiers.

We also provide a utility [`graphein.protein.utils.download_alphafold_structure()`](https://graphein.ai/modules/graphein.protein.html#graphein.protein.utils.download_alphafold_structure) to download PDBs and associated metadata (aligned error scores as an accompanying `JSON` file). There is also an [equivalent function](https://graphein.ai/modules/graphein.protein.html#graphein.protein.utils.download_pdb) for downloading structures from RCSB. After downloading the structure, simply use Graphein to compute the graph as you normally would. You can check out the [`residue_graphs.ipynb`](https://graphein.ai/notebooks/residue_graphs.html). notebook tutorial to go deeper into protein graph construction utilities in Graphein.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-r-j/graphein/blob/master/notebooks/alphafold_protein_graph_tutorial.ipynb)

## Quickstart

The structures are accessed via their [UniProt](https://www.uniprot.org/) accession codes.

Here, we'll be looking at [`Q8W3K0`](https://alphafold.ebi.ac.uk/entry/Q8W3K0), a rather beautiful model of a protein with a not-so-catchy name: Probable disease resistance protein At1g58602.

In [1]:
# Install graphein if necessary
#!pip install graphein

In [2]:
import graphein.protein as gp

g = gp.construct_graph(uniprot_id="Q8W3K0")
gp.plotly_protein_structure_graph(g, colour_nodes_by="plddt") # Colouring by plDDT shows the assigned confidences!

Reading PDB file...         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Processing PDB dataframe... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Initializing graph...       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

## Downloading Structures
First, we need to download the strutures from AlphaFoldDB. One can download either the structure alone or together the aligned scores as a JSON.


In [4]:
from graphein.protein.utils import download_alphafold_structure

# Download the PDB file for an example protein (UniProt: Q8W3K0) with the aligned score
protein_path = download_alphafold_structure("Q8W3K0", out_dir = "/tmp", aligned_score=True)
protein_path

('/tmp/Q8W3K0.pdb', '/tmp/Q8W3K0.json')

In [5]:
# Download the PDB file for an example protein (UniProt: Q8W3K0) without the aligned score
protein_path = download_alphafold_structure("Q8W3K0", out_dir="/tmp", aligned_score=False)
protein_path

'/tmp/Q8W3K0.pdb'

## Constructing the Graph
We show a simplified workflow here. For more information and a fuller exposition of the features, please refer to the [Residue Graph Tutorial](https://graphein.ai/notebooks/residue_graphs.html).

First, we require a [config](https://graphein.ai/modules/graphein.protein.html#graphein.protein.config.ProteinGraphConfig) object.

In [6]:
# Load the default config
c = gp.ProteinGraphConfig(granularity='CA')

# Construct the graph!
g = gp.construct_graph(pdb_path=protein_path)

# Plot
gp.plotly_protein_structure_graph(g, node_size_multiplier=0.5, colour_nodes_by="residue_name")

Reading PDB file...         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Processing PDB dataframe... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Initializing graph...       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

## Adding Edges
What if we want to add some more edges to the graph? Well..

[Documentation for edge funcs](https://graphein.ai/modules/graphein.protein.html#edges)

In [7]:
config = gp.ProteinGraphConfig(edge_construction_functions=[
                                gp.add_aromatic_interactions,
                                gp.add_cation_pi_interactions,
                                gp.add_hydrophobic_interactions,
                                gp.add_ionic_interactions
                                ]
)

g = gp.construct_graph(pdb_path=protein_path, config=config)

gp.plotly_protein_structure_graph(g, colour_edges_by="kind", colour_nodes_by="residue_name", label_node_ids=False, node_size_multiplier=2, node_size_min=5)

Reading PDB file...         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Processing PDB dataframe... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Initializing graph...       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Constructing edges...       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

You can also check out our [Residue Graph Tutorial](https://graphein.ai/notebooks/residue_graphs.html) for more customisation options!! For more details on visualisation, please refer to the [interactive visualisation tutorial](https://graphein.ai/notebooks/interactive_plotlY_examples.html) or the [documentation](https://graphein.ai/modules/graphein.protein.html#module-graphein.protein.visualisation).